# Todo
* Sensitivity 250 500 1000
* For the 500, take top 10% heightest weight
* Take that threshold for other topics


In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#import tempfile
#TEMP_FOLDER = tempfile.gettempdir()
#print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('retina')

from sklearn import preprocessing

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.test.utils import datapath

In [ ]:
# The model name we want to load
model_name = "matrix1_500topic"

current_path = os.getcwd()
current_path = Path(current_path)
load_path = current_path.parent / "./Data/LDAModel/{}/{}".format(model_name,model_name)
load_path = str(load_path)

In [ ]:
load_path

In [ ]:
matrix_1 = gensim.models.ldamulticore.LdaMulticore.load(load_path)

In [ ]:
matrix_1.print_topics()